In [1]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
#!pip install dask[dataframe]
#!pip install pandas --upgrade
# We have to do this as dask is bad at guessing the types of objects for some reason
# Also, if you are having trouble with running the dataframe and continuously get 
# Fatal Python error: GC object already tracked, try to update pandas
data_types = {
    'ID':  np.int64,
    'Case Number':  object,
    'Date': object,
    'Block': object,
    'IUCR': object,
    'Primary Type': object,
    'Description':  object,
    'Location Description': object,
    'Arrest':  bool,
    'Domestic': bool,
    'Beat': np.int64,
    'District': np.float64,
    'Ward': np.float64,
    'Community Area': np.float64,
    'FBI Code': object,
    'X Coordinate': np.float64,
    'Y Coordinate': np.float64,
    'Year': np.int64,
    'Updated On': object,
    'Latitude': np.float64,
    'Longitude': np.float64,
    'Location': object
}
df = pd.read_csv('crimes.csv')
df.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10446260,HZ185168,3/12/2016 23:50:00,078XX S YATES BLVD,486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,7,43,08B,NaN,NaN,2016,3/19/2016 15:45:00,NaN,NaN,NaN
1,10446329,HZ185174,3/12/2016 23:50:00,034XX N HALSTED ST,460,BATTERY,SIMPLE,STREET,False,False,...,44,6,08B,NaN,NaN,2016,3/19/2016 15:45:00,NaN,NaN,NaN
2,10446859,HZ185472,3/12/2016 23:50:00,006XX S CLARK ST,460,BATTERY,SIMPLE,RESIDENCE,False,False,...,2,32,08B,NaN,NaN,2016,3/19/2016 15:45:00,NaN,NaN,NaN
3,10446323,HZ185185,3/12/2016 23:50:00,078XX S ABERDEEN ST,497,BATTERY,AGGRAVATED DOMESTIC BATTERY: OTHER DANG WEAPON,APARTMENT,True,True,...,17,71,04B,NaN,NaN,2016,3/19/2016 15:45:00,NaN,NaN,NaN
4,10446385,HZ185156,3/12/2016 23:45:00,068XX S JUSTINE ST,031A,ROBBERY,ARMED: HANDGUN,SIDEWALK,False,False,...,17,67,3,NaN,NaN,2016,3/19/2016 15:45:00,NaN,NaN,NaN


In [2]:
from geopy.geocoders import Nominatim
# Need to install geopy
#!pip install geopy
geolocator = Nominatim()
print df['Block'][0]
location = geolocator.geocode(df['Block'][0])
print location.latitude, location.longitude
# print df['Latitude'][0], data_subset['Longitude'][0]

078XX S YATES BLVD
41.766052 -87.5665077


In [3]:
poverty = pd.read_csv('poverty.csv')
poverty.head()


,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX,Neighborhoods,Zip Codes
0,1,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39,Albany Park,60625
1,2,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46,Altgeld Gardens,60827
2,3,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20,Andersonville,60640
3,4,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17,Arcadia Terrace,60659
4,5,North Center,0.3,7.5,5.2,4.5,26.2,57123,6,Archer Heights,60632


In [ ]:
#sub = df[:10]

# Gets full address using block address with "Chicago" appended
# Extracts zip code from full address
#
def getZip(row):
    address = geolocator.geocode(row['Block'] + " Chicago").address.replace(',', "")
    return next(iter([int(s) for s in address.split() if s.isdigit()]), 0)

#sub.head()
df['Zip Codes'] = df.apply(getZip, axis=1, reduce=False)
df.head()